In [184]:
## Import Libraries
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
from copy import deepcopy

In [185]:
##Import data
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')
ftd.dtype.names = ('img_id', 'description')

In [141]:
## Function- to convert sring into list.
## First convert strings to list of rows equal to rows in fe by making a function
## All image ids are one string in each row. By converting this string into list we will make each image ID an item in the list.

def mk_list(str_vector):
    des_byid = str_vector
    all_x = []
    for i in des_byid:
        j = i.split()
        #if len(j) > 0:
        all_x.append(j)

    return all_x


In [142]:
## convert strings in each row of pre-que to list of itmens.
## each row of all_x contains a list of image id(S) before question is asked and action like elimination/selection is done.

all_x = mk_list(fe['pre_que'])


29


In [143]:
##check blanks
## As there are some rows where there is only one image or blank/erronous question. We check that and save those indices.
## create blank list
#good_index = ['True']*len(all_x)

bl =[]

for img,q in zip(enumerate(all_x), mk_list(qa['que'])):
    if (len(img[1]) <2 or len(q) < 2) and img[0] not in bl:
        bl.append(img[0])


[10, 43, 114, 115, 117, 118, 130, 131, 134, 135, 136, 137, 138, 139, 140, 234, 358, 408, 537, 607, 636, 674, 923, 928, 929, 931, 932, 949, 1078, 1079, 1196, 1250, 1251, 1258, 1264, 1274, 1364, 1433, 1493, 1494, 1545, 1600, 1753, 1907, 1996, 2045, 2077, 2095, 2103, 2105, 2106, 2167, 2168, 2169, 2201, 2308, 2312, 2413, 2455, 2535, 2536, 2542, 2657, 2681, 2706, 2772, 2773, 2774, 2778, 2779, 2781, 2782, 2783, 2785, 2786, 2787, 2788, 2789, 2790, 2791, 2798, 2873, 2876, 2902, 2942, 2949, 2965, 3079, 3201, 3207, 3258, 3308, 3411, 3416, 3426, 3439, 3440, 3444, 3445, 3446, 3449, 3496, 3506, 3507, 3549, 3600, 3602, 3610, 3612, 3616, 3619, 3621, 3624, 3626, 3628, 3652, 3654, 3655, 3656, 3665, 3682, 3715, 3726, 3727, 3730, 3733, 3735, 3829, 3912, 3954, 3959, 4001, 4002, 4003, 4004, 4005, 4006, 4007, 4008, 4024, 4025, 4302, 4335, 4363, 4415, 4450, 4522, 4523, 4536, 4572, 4621, 4723, 4734, 4736, 4737, 4792, 4793, 4794, 4823, 4824, 4825, 4826, 4827, 4828, 4829, 4830, 4898, 4904, 4916, 4989, 5019, 504

In [146]:
## Now for each row in the list above, find description and add it
all_i_txt1 = []
original_ind = []
bl_ind = []

for rown in range(len(all_x)):
    if rown in bl:
        bl_ind.append(rown)
    else:
        #print(all_x.index)
        int_i_txt = []
        for img in all_x[rown]:
            if img in ftd['img_id']:
                d = str(ftd['description'][list(ftd['img_id']).index(img)])
                int_i_txt.append(str(d))
        original_ind.append(rown)
        all_i_txt1.append(int_i_txt)
        
            
            

In [148]:
## add question and answer to each data point

all_i_txt = [row + [qa['que'][rn].strip(), qa['ans'][rn].strip()] for rn, row in zip(original_ind, all_i_txt1)]

    
    
    

In [151]:

## model
cv = CountVectorizer(ngram_range= (2,2))

In [152]:
## all similarities
all_sim = []
for l in range(len(all_i_txt)):
    H = cv.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim.append(sim)

In [153]:
#similarities[-2] meaning taking only the array for question
all_sim_q = []
for i in range(len(all_sim)):
    all_sim_q.append(all_sim[i][-2][:-2])
print(len(all_sim_q))


14644


In [154]:
## Sort all the above similarities with argsort which gives indices
sorted_ind = []
for si in all_sim_q:
    y_top = si.argsort()[::-1]
    sorted_ind.append(y_top)


In [155]:
## with above indices, create new sorted array per que, this is done to then find max step difference 
sorted_sim_all = []
for q,w in zip(sorted_ind, all_sim_q):
    sorted_sim = []
    for e in q:
        sorted_sim.append(w[e])
    sorted_sim_all.append(sorted_sim)


14644


In [156]:
##find the difference between two similarity score in each array.
sim_dif_all = []

for d in range(len(sorted_sim_all)):
    sim_dif = [sorted_sim_all[d][f-1] - sorted_sim_all[d][f] for f in range(1, len(sorted_sim_all[d]))]
    sim_dif_all.append(sim_dif)


14644

In [157]:
## Find maximum difference in similarity scores per row and also find the indice where to cut-off

max_dif = []
max_dif_ind =[]
for g in range(len(sim_dif_all)):
    
    maxi= max(sim_dif_all[g])
    max_dif.append(maxi)
    max_dif_ind.append(sim_dif_all[g].index(maxi))


In [159]:
## Cut off sorted indices till maximum difference index
sorted_ind_lim = [indi[0:max_indi+1] for indi,max_indi in zip(sorted_ind, max_dif_ind)]


In [160]:
##clean_all_x- Without outliers
all_x_clean = [all_x[i] for i in original_ind]


In [162]:
top_img = zip(all_x_clean, sorted_ind_lim)

In [163]:
## convert to img_id from document index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred

In [164]:
pred_img_elim = mk_imgid(top_img)

In [166]:
pred_img_nonelim = [set(i).difference(set(j)) for i, j in zip(all_x_clean, pred_img_elim)]

In [109]:
## This we need to know actually eliminated images so as to evaluate our model performance
all_cheat1 = mk_list(fe['curr_elim'])
all_cheat = [all_cheat1[i] for i in original_ind]

In [171]:
tp = [len(set(a).intersection(set(p))) for a,p in zip(all_cheat, pred_img_elim)]
fn = [len(set(a).intersection(set(p))) for a,p in zip(all_cheat, pred_img_nonelim)]

In [177]:
pre_all= [tp/len(set(p)) if len(set(p)) > 0 else 0 for tp,p in zip(tp, pred_img_elim)]
rec_all = [tp/(tp + fn) if tp+fn > 0 else 0 for tp,fn in zip(tp, fn)]



In [179]:
fs_all= [(0.5*0.5 + 1)*pr*re/ (0.5*0.5*pr + re) if pr > 0 or re >0 else 0 for pr, re in zip(pre_all, rec_all)]

In [181]:
pre_mean = np.mean(np.array(pre_all))
rec_mean = np.mean(np.array(rec_all))
fs_mean = np.mean(np.array(fs_all))

In [117]:
op1 = ["2", "CV", "ngram_range 2,2", "Cosinesimilarity and ranking difference",str(round(pre_mean,2)), str(round(rec_mean,2)), str(round(fs_mean,2))]

In [118]:
with open('Results with description.txt', 'a') as f:
    f.write('; '.join(op1) + '\n')